In [ ]:
!pip install --upgrade gensim
!pip install transformers
!pip install -U sentence-transformers
!pip install pytorch-lightning==1.1.0 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ast
from collections import Counter
import re

import nltk
import numpy as np
from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures
nltk.download('stopwords')
nltk.download('punkt')

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import cohen_kappa_score, accuracy_score, f1_score, roc_auc_score, precision_score, classification_report, hamming_loss
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight

from tabulate import tabulate

from gensim.models import Word2Vec

from numba import cuda

from sentence_transformers import SentenceTransformer

import pytorch_lightning as pl
from pytorch_lightning.metrics.functional.classification import auroc
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, Dataset
from torch.autograd import Variable

from transformers import AutoModel, BertTokenizerFast, FeatureExtractionPipeline # BertTokenizer, BertModel, BertForSequenceClassification}
# optimizer from hugging face transformers
from transformers import AdamW, get_linear_schedule_with_warmup

In [ ]:
# cuda.select_device(0)
# cuda.close()

## Cosas BETO

In [ ]:
device = torch.device("cuda") 

tokenizer = BertTokenizerFast.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")
# def custom_tokenizer(text):
#   return tokenizer(text, truncation=True, max_length=500)
# model = AutoModel.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased", return_dict=True)

# # freeze all the parameters
# for param in model.parameters():
#     param.requires_grad = False

# model.eval()
# BETO_features = FeatureExtractionPipeline(model, custom_tokenizer, device=0)

In [ ]:
text = ["hola amiguitos", "bienvenidos a mi casa compañeros"]
sent_id = tokenizer.batch_encode_plus(text, padding=True, return_token_type_ids=False)
print(sent_id)

In [ ]:
!nvidia-smi

## Carga de datos

In [ ]:
categories_number_words = {
        1: "Apoyo Pedagógico en asignaturas",
        3: "Apoyo pedagógico personal",
        4: "Tutoría entre pares",
        7: "Hacer a la familia partícipe del proceso",
        8: "Apoyo psicóloga(o)",
        9: "Apoyo fonoaudióloga(o)",
        10: "Apoyo Educador(a) Diferencial",
        11: "Apoyo Kinesióloga(o)",
        12: "Apoyo Médico General",
        13: "Apoyo Terapeuta Ocupacional",
        14: "Control Neurólogo",
        15: "Apoyo Interdisciplinario",
        16: "Adecuación curricular de acceso",
        17: "Adecuación curricular de objetivos"
    }
categories_words_number = {v: k for k, v in categories_number_words.items()}

diagnoses_codes = {
    "Trastorno específico del lenguaje": 0,
    "Trastorno por déficit atencional": 1,
    "Dificultad específica de aprendizaje": 2,
    "Discapacidad intelectual": 3,
    "Discapacidad visual": 4,
    "Trastorno del espectro autista": 5,
    "Discapacidad auditiva - Hipoacusia": 6,
    "Funcionamiento intelectual limítrofe": 7,
    "Síndrome de Down": 8,
    "Trastorno motor": 9,
    "Multidéficit": 10,
    "Retraso global del desarrollo": 11
}

diagnoses_keys = list(diagnoses_codes.keys())

def transform_diag_to_array(code):
    arr = np.zeros(len(diagnoses_keys), dtype=int)
    for (index, label) in enumerate(diagnoses_keys):
        if diagnoses_codes[label]==code:
            arr[index] = 1
    return arr

### Datos pre cargados

In [ ]:
# students_strats = pd.read_csv('/home/jamunoz/datasets/anonimized_dataset.csv')
# columns = students_strats.columns
# for var in columns:
#     if var != 'Diagnoses' and var != 'Index':
#         students_strats[var] = students_strats[var].apply(ast.literal_eval)
# students_strats.columns
train_dataset = pd.read_csv('/home/jamunoz/datasets/train_ds.csv', keep_default_na=False)
val_dataset = pd.read_csv('/home/jamunoz/datasets/val_ds.csv', keep_default_na=False)
test_dataset = pd.read_csv('/home/jamunoz/datasets/test_ds.csv', keep_default_na=False)

# Add OHE diagnosis
train_OHE_diags = []
for diag in train_dataset['Encoded Diagnosis']:
    train_OHE_diags.append(transform_diag_to_array(diag))
temp_train_diags_df = pd.DataFrame(train_OHE_diags, columns=diagnoses_keys)
train_dataset = pd.concat([train_dataset, temp_train_diags_df], axis=1)

val_OHE_diags = []
for diag in val_dataset['Encoded Diagnosis']:
    val_OHE_diags.append(transform_diag_to_array(diag))
temp_val_diags_df = pd.DataFrame(val_OHE_diags, columns=diagnoses_keys)
val_dataset = pd.concat([val_dataset, temp_val_diags_df], axis=1)

test_OHE_diags = []
for diag in test_dataset['Encoded Diagnosis']:
    test_OHE_diags.append(transform_diag_to_array(diag))
temp_test_diags_df = pd.DataFrame(test_OHE_diags, columns=diagnoses_keys)
test_dataset = pd.concat([test_dataset, temp_test_diags_df], axis=1)

In [ ]:
train_dataset.head()

## Experimentos

In [ ]:
# y_keys = list(strat_present.keys())
Y_KEYS = list(categories_words_number.keys())

# df = pd.DataFrame(data=new_dataset_to_export)
# X = df
# Y = df[y_keys]
X_train = train_dataset.drop(Y_KEYS, axis=1)
Y_train = train_dataset[Y_KEYS]
X_val = val_dataset.drop(Y_KEYS, axis=1)
Y_val = val_dataset[Y_KEYS]

strats_amounts = {
              'Adecuación curricular de acceso': 2264,
              'Hacer a la familia partícipe del proceso': 2048,
              'Apoyo Interdisciplinario': 1441, 
              'Apoyo Educador(a) Diferencial': 1311,
              'Apoyo pedagógico personal': 1240,
              'Apoyo fonoaudióloga(o)': 378,
              'Apoyo psicóloga(o)': 588,
              'Apoyo Terapeuta Ocupacional': 153,
              'Tutoría entre pares': 350,
              'Control Neurólogo': 63,
              'Apoyo Médico General': 64,
              'Apoyo Kinesióloga(o)': 32,
              'Adecuación curricular de objetivos': 281,
              'Apoyo Pedagógico en asignaturas': 1314
}
most_unbalanced_strategies = [strategy for strategy in Y_KEYS if (
    strats_amounts[strategy] < (len(X_train) + len(X_val))*0.15 or strats_amounts[strategy] > (len(X_train) + len(X_val))*0.85)]
less_unbalanced_strategies = [strategy for strategy in Y_KEYS if strategy not in most_unbalanced_strategies]
only_one_strat = [Y_KEYS[0]]

In [ ]:
X_train.keys()

In [ ]:
percs = X_train['All perceptions']
seq_len = [len(i.split()) for i in percs]

pd.Series(seq_len).hist(bins = 30)

In [ ]:
print(0 in seq_len)

In [ ]:
percs = X_train['Special Education Teacher Perceptions']
seq_len = [len(i.split()) for i in percs]

pd.Series(seq_len).hist(bins = 30)

In [ ]:
print(0 in seq_len)

In [ ]:
percs = X_train['Speech Therapist Perceptions']
seq_len = [len(i.split()) for i in percs]

pd.Series(seq_len).hist(bins = 30)

In [ ]:
percs = X_train['Psychologist Perceptions']
seq_len = [len(i.split()) for i in percs]

pd.Series(seq_len).hist(bins = 30)

In [ ]:
percs = X_train['Medical Perceptions']
seq_len = [len(i.split()) for i in percs]

pd.Series(seq_len).hist(bins = 30)

In [ ]:
# sentence_model = SentenceTransformer('xlm-r-bert-base-nli-stsb-mean-tokens', device='cuda')

In [ ]:
# sentence_model.max_seq_length = 500

In [ ]:
# sentence_model.encode(X['All perceptions'][0])

In [ ]:
!nvidia-smi

## Transformación de datos

In [ ]:
# class JoinedProfessionalPerceptionsDataset(Dataset):
    
#     def __init__(self, data: pd.DataFrame, tokenizer: BertTokenizerFast, max_token_len=200):
#         self.data = data
#         self.tokenizer = tokenizer
#         self.max_token_len = max_token_len
        
#     def __len__(self):
#         return len(self.data)
    
#     def __getitem__(self, index: int):
        
#         data_row = self.data.iloc[index]
        
#         text = data_row['All perceptions']    
#         labels =  data_row[Y_KEYS]
        
#         encoding = self.tokenizer.encode_plus(
#             text,
#             # add_special_tokens=True,
#             max_length=self.max_token_len,
#             padding='max_length',
#             truncation=True,
#             return_token_type_ids=False,
#             return_attention_mask=True,
#             return_tensors='pt'
#         )
        
#         return dict(
#             text=text,
#             labels=torch.FloatTensor(labels),
#             # posiblemente cambiar porque el modelo recibe el tensor agrupado
#             input_ids=encoding["input_ids"].flatten(),
#             attention_mask=encoding["attention_mask"].flatten()
#         )

In [ ]:
class DiagnosisAndDividedPerceptionsDataset(Dataset):
    
    def __init__(self, data: pd.DataFrame, tokenizer: BertTokenizerFast, max_token_len=200):
        self.data = data
        self.tokenizer = tokenizer
        self.max_token_len = max_token_len
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index: int):
        
        data_row = self.data.iloc[index]
        
        sne_teacher_obs = data_row['Special Education Teacher Perceptions']
        speech_therapist_obs = data_row['Speech Therapist Perceptions']
        psychologist_obs = data_row['Psychologist Perceptions']
        medical_obs = data_row['Medical Perceptions']
        
        labels =  data_row[Y_KEYS]
        
        encoding_sne_obs = self.tokenizer.encode_plus(
            sne_teacher_obs,
            # add_special_tokens=True,
            max_length=self.max_token_len,
            padding='max_length',
            truncation=True,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        encoding_speech_obs = self.tokenizer.encode_plus(
            speech_therapist_obs,
            # add_special_tokens=True,
            max_length=self.max_token_len,
            padding='max_length',
            truncation=True,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        encoding_psychologist_obs = self.tokenizer.encode_plus(
            psychologist_obs,
            # add_special_tokens=True,
            max_length=self.max_token_len,
            padding='max_length',
            truncation=True,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        encoding_medical_obs = self.tokenizer.encode_plus(
            medical_obs,
            # add_special_tokens=True,
            max_length=self.max_token_len,
            padding='max_length',
            truncation=True,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        encoding_diagnosis = self.tokenizer.encode_plus(
            medical_obs,
            # add_special_tokens=True,
            max_length=30,
            padding='max_length',
            truncation=True,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        dict_to_return = dict(
            sne_text=sne_teacher_obs,
            st_text=speech_therapist_obs,
            p_text=psychologist_obs,
            m_text=medical_obs,
            labels=torch.FloatTensor(labels),
            # posiblemente cambiar porque el modelo recibe el tensor agrupado
            sne_input_ids=encoding_sne_obs["input_ids"].flatten(),
            sne_attention_mask=encoding_sne_obs["attention_mask"].flatten(),
            st_input_ids=encoding_speech_obs["input_ids"].flatten(),
            st_attention_mask=encoding_speech_obs["attention_mask"].flatten(),
            p_input_ids=encoding_psychologist_obs["input_ids"].flatten(),
            p_attention_mask=encoding_psychologist_obs["attention_mask"].flatten(),
            m_input_ids=encoding_medical_obs["input_ids"].flatten(),
            m_attention_mask=encoding_medical_obs["attention_mask"].flatten(),
            diagnosis=data_row[diagnoses_keys].to_numpy(dtype="int"),
            diagnosis_text=data_row['Diagnosis'],
            diagnosis_ids=encoding_diagnosis["input_ids"].flatten(),
            diagnosis_attention_mask=encoding_diagnosis["attention_mask"].flatten(),
        )
        
        return dict_to_return

In [ ]:
trans_sample = DiagnosisAndDividedPerceptionsDataset(train_dataset, tokenizer)
sample_item = trans_sample[0]
sample_item['diagnosis']

In [ ]:
# transformed_train_ds = JoinedProfessionalPerceptionsDataset(train_dataset, tokenizer)

In [ ]:
# sample_item = transformed_train_ds[0]

In [ ]:
# sample_item["text"]

In [ ]:
# sample_item["labels"]

In [ ]:
# sample_item["input_ids"].size()

In [ ]:
# class JoinedProfessionalPerceptionsDataModule(pl.LightningDataModule):
    
#     def __init__(self, train_df, val_df, test_df, tokenizer, batch_size=8, max_token_len=200):
#         super().__init__()
#         self.train_df = train_df
#         self.val_df = val_df
#         self.test_df = test_df
#         self.tokenizer = tokenizer
#         self.batch_size = batch_size
#         self.max_token_len = max_token_len
        
#     def setup(self):
#         self.train_dataset = JoinedProfessionalPerceptionsDataset(
#             self.train_df,
#             self.tokenizer,
#             self.max_token_len
#         )
        
#         self.val_dataset = JoinedProfessionalPerceptionsDataset(
#             self.val_df,
#             self.tokenizer,
#             self.max_token_len
#         )
        
#         self.test_dataset = JoinedProfessionalPerceptionsDataset(
#             self.test_df,
#             self.tokenizer,
#             self.max_token_len
#         )
        
#     def train_dataloader(self):
#         return DataLoader(
#             self.train_dataset,
#             batch_size=self.batch_size,
#             shuffle=True,
#             num_workers=4
#         )
    
#     def val_dataloader(self):
#         return DataLoader(
#             self.val_dataset,
#             batch_size=1,
#             num_workers=4
#         )
    
#     def test_dataloader(self):
#         return DataLoader(
#             self.test_dataset,
#             batch_size=1,
#             num_workers=4
#         )

In [ ]:
class DiagnosisAndDividedPerceptionsDataModule(pl.LightningDataModule):
    
    def __init__(self, train_df, val_df, test_df, tokenizer, batch_size=8, max_token_len=200):
        super().__init__()
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df
        self.tokenizer = tokenizer
        self.batch_size = batch_size
        self.max_token_len = max_token_len
        
    def setup(self):
        self.train_dataset = DiagnosisAndDividedPerceptionsDataset(
            self.train_df,
            self.tokenizer,
            self.max_token_len
        )
        
        self.val_dataset = DiagnosisAndDividedPerceptionsDataset(
            self.val_df,
            self.tokenizer,
            self.max_token_len
        )
        
        self.test_dataset = DiagnosisAndDividedPerceptionsDataset(
            self.test_df,
            self.tokenizer,
            self.max_token_len
        )
        
    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=4
        )
    
    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=1,
            num_workers=4
        )
    
    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=1,
            num_workers=4
        )

In [ ]:
N_EPOCHS = 10
BATCH_SIZE = 2
# special batch size when using 500 tokens limit
SPECIAL_BATCH_SIZE = 1

# data_module = JoinedProfessionalPerceptionsDataModule(train_dataset, val_dataset, test_dataset, tokenizer, batch_size=BATCH_SIZE)
# data_module.setup()
# Tried to use different BERT for each perception type but was unable due to run out of memory

# data_module_100 = DiagnosisAndDividedPerceptionsDataModule(
#     train_dataset, 
#     val_dataset, 
#     test_dataset, 
#     tokenizer, 
#     batch_size=BATCH_SIZE,
#     max_token_len=100
# )
# data_module_100.setup()


data_module_200 = DiagnosisAndDividedPerceptionsDataModule(
    train_dataset, 
    val_dataset, 
    test_dataset, 
    tokenizer, 
    batch_size=BATCH_SIZE,
    max_token_len=200
)
data_module_200.setup()

data_module_300 = DiagnosisAndDividedPerceptionsDataModule(
    train_dataset, 
    val_dataset, 
    test_dataset, 
    tokenizer, 
    batch_size=BATCH_SIZE,
    max_token_len=300
)
data_module_300.setup()

data_module_400 = DiagnosisAndDividedPerceptionsDataModule(
    train_dataset, 
    val_dataset, 
    test_dataset, 
    tokenizer, 
    batch_size=SPECIAL_BATCH_SIZE,
    max_token_len=400
)
data_module_400.setup()

data_module_500 = DiagnosisAndDividedPerceptionsDataModule(
    train_dataset, 
    val_dataset, 
    test_dataset, 
    tokenizer, 
    batch_size=SPECIAL_BATCH_SIZE,
    max_token_len=500
)
data_module_500.setup()

mini_200_module = DiagnosisAndDividedPerceptionsDataModule(
    train_dataset, 
    val_dataset, 
    test_dataset, 
    tokenizer, 
    batch_size=SPECIAL_BATCH_SIZE,
    max_token_len=200
)
mini_200_module.setup()

mini_300_module = DiagnosisAndDividedPerceptionsDataModule(
    train_dataset, 
    val_dataset, 
    test_dataset, 
    tokenizer, 
    batch_size=SPECIAL_BATCH_SIZE,
    max_token_len=300
)
mini_300_module.setup()

mini_400_module = DiagnosisAndDividedPerceptionsDataModule(
    train_dataset, 
    val_dataset, 
    test_dataset, 
    tokenizer, 
    batch_size=SPECIAL_BATCH_SIZE,
    max_token_len=400
)
mini_400_module.setup()

mini_500_module = DiagnosisAndDividedPerceptionsDataModule(
    train_dataset, 
    val_dataset, 
    test_dataset, 
    tokenizer, 
    batch_size=SPECIAL_BATCH_SIZE,
    max_token_len=500
)
mini_500_module.setup()

## Modeling

### Evaluation

In [ ]:
# class JoinedObservationsClassifier(pl.LightningModule):
    
#     def __init__(self, n_classes: int, steps_per_epoch=None, n_epochs=None):
#         super().__init__()
#         self.bert = AutoModel.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased", return_dict=True)
#         self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)
#         self.steps_per_epoch = steps_per_epoch
#         self.n_epochs = n_epochs
#         self.criterion = nn.BCELoss()
        
#     def forward(self, inputs_ids, attention_mask, labels=None):
#         output = self.bert(inputs_ids, attention_mask=attention_mask)
#         output = self.classifier(output.pooler_output)
#         output = torch.sigmoid(output)
#         loss = 0
#         if labels is not None:
#             loss = self.criterion(output, labels)
#         return loss, output
    
#     def training_step(self, batch, batch_idx):
#         input_ids = batch['input_ids']
#         attention_mask = batch['attention_mask']
#         labels = batch['labels']
#         loss, outputs = self(input_ids, attention_mask, labels)
#         self.log("train loss", loss, prog_bar=True, logger=True)
#         return {'loss': loss, 'predictions': outputs, 'labels': labels}
    
#     def validation_step(self, batch, batch_idx):
#         input_ids = batch['input_ids']
#         attention_mask = batch['attention_mask']
#         labels = batch['labels']
#         loss, outputs = self(input_ids, attention_mask, labels)
#         self.log("val loss", loss, prog_bar=True, logger=True)
#         return loss
    
#     def test_step(self, batch, batch_idx):
#         input_ids = batch['input_ids']
#         attention_mask = batch['attention_mask']
#         labels = batch['labels']
#         loss, outputs = self(input_ids, attention_mask, labels)
#         self.log("test loss", loss, prog_bar=True, logger=True)
#         return loss
    
#     def training_epoch_end(self, outputs):
#         labels = []
#         predictions = []
        
#         for output in outputs:
#             for out_labels in output['labels'].detach().cpu():
#                 labels.append(out_labels)
                
#             for out_predictions in output['predictions'].detach().cpu():
#                 predictions.append(out_predictions)
                
#         labels = torch.stack(labels)
#         predictions = torch.stack(predictions)
        
#         for i, name in enumerate(Y_KEYS):
#             roc_score = auroc(predictions[:, i], labels[:, i])
#             self.logger.experiment.add_scalar(f"{name}__roc_auc/Train", roc_score, self.current_epoch)
            
#     def configure_optimizers(self): 
#         optimizer = AdamW(self.parameters(), lr=2e-5)
        
#         warmup_steps = self.steps_per_epoch // 3
#         total_steps = self.steps_per_epoch * self.n_epochs - warmup_steps
        
#         scheduler = get_linear_schedule_with_warmup(
#             optimizer,
#             warmup_steps,
#             total_steps
#         )
        
#         return [optimizer], [scheduler] 

In [ ]:
class DiagnosisAndDividedObservationsClassifier(pl.LightningModule):
    
    def __init__(self, n_classes: int, n_diags: int, steps_per_epoch=None, n_epochs=None):
        super().__init__()
        self.bert = AutoModel.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased", return_dict=True)
        # self.diagnosis_layer = nn.Linear(1, 10)
        self.classifier = nn.Linear((self.bert.config.hidden_size*4)+n_diags, n_classes)
        self.steps_per_epoch = steps_per_epoch
        self.n_epochs = n_epochs
        self.criterion = nn.BCELoss()
        
    def forward(self, 
                sne_inputs_ids, sne_attention_mask,
                st_inputs_ids, st_attention_mask,
                p_inputs_ids, p_attention_mask,
                m_inputs_ids, m_attention_mask,
                diagnosis, labels=None):
        # print(diagnosis)
        output_sne = self.bert(sne_inputs_ids, attention_mask=sne_attention_mask)
        output_st = self.bert(st_inputs_ids, attention_mask=st_attention_mask)
        output_p = self.bert(p_inputs_ids, attention_mask=p_attention_mask)
        output_m = self.bert(m_inputs_ids, attention_mask=m_attention_mask)
        # output_diagnosis = self.diagnosis_layer(torch.tensor([int(diagnosis)]))
        # diags_tensor = torch.from_numpy(diagnosis)
        output = torch.cat(
            (output_sne.pooler_output,
             output_st.pooler_output,
             output_p.pooler_output,
             output_m.pooler_output,
             diagnosis #_tensor,
            ), dim=1)
        output = self.classifier(output)
        output = torch.sigmoid(output)
        loss = 0
        if labels is not None:
            loss = self.criterion(output, labels)
        return loss, output
    
    def training_step(self, batch, batch_idx):
        sne_input_ids = batch['sne_input_ids']
        sne_attention_mask = batch['sne_attention_mask']
        st_input_ids = batch['st_input_ids']
        st_attention_mask = batch['st_attention_mask']
        p_input_ids = batch['p_input_ids']
        p_attention_mask = batch['p_attention_mask']
        m_input_ids = batch['m_input_ids']
        m_attention_mask = batch['m_attention_mask']
        diagnosis = batch['diagnosis']
        labels = batch['labels']
        loss, outputs = self(
            sne_input_ids, sne_attention_mask,
            st_input_ids, st_attention_mask,
            p_input_ids, p_attention_mask,
            m_input_ids, m_attention_mask,
            diagnosis, labels)
        self.log("train loss", loss, prog_bar=True, logger=True)
        return {'loss': loss, 'predictions': outputs, 'labels': labels}
    
    def validation_step(self, batch, batch_idx):
        sne_input_ids = batch['sne_input_ids']
        sne_attention_mask = batch['sne_attention_mask']
        st_input_ids = batch['st_input_ids']
        st_attention_mask = batch['st_attention_mask']
        p_input_ids = batch['p_input_ids']
        p_attention_mask = batch['p_attention_mask']
        m_input_ids = batch['m_input_ids']
        m_attention_mask = batch['m_attention_mask']
        diagnosis = batch['diagnosis']
        labels = batch['labels']
        loss, outputs = self(
            sne_input_ids, sne_attention_mask,
            st_input_ids, st_attention_mask,
            p_input_ids, p_attention_mask,
            m_input_ids, m_attention_mask,
            diagnosis, labels)
        self.log("val loss", loss, prog_bar=True, logger=True)
        return loss
    
    def test_step(self, batch, batch_idx):
        sne_input_ids = batch['sne_input_ids']
        sne_attention_mask = batch['sne_attention_mask']
        st_input_ids = batch['st_input_ids']
        st_attention_mask = batch['st_attention_mask']
        p_attention_mask = batch['p_attention_mask']
        m_input_ids = batch['m_input_ids']
        m_attention_mask = batch['m_attention_mask']
        diagnosis = batch['diagnosis']
        labels = batch['labels']
        loss, outputs = self(
            sne_input_ids, sne_attention_mask,
            st_input_ids, st_attention_mask,
            p_input_ids, p_attention_mask,
            m_input_ids, m_attention_mask,
            diagnosis, labels)
        self.log("test loss", loss, prog_bar=True, logger=True)
        return loss
    
    def training_epoch_end(self, outputs):
        labels = []
        predictions = []
        
        for output in outputs:
            for out_labels in output['labels'].detach().cpu():
                labels.append(out_labels)
                
            for out_predictions in output['predictions'].detach().cpu():
                predictions.append(out_predictions)
                
        labels = torch.stack(labels)
        predictions = torch.stack(predictions)
        
        for i, name in enumerate(Y_KEYS):
            roc_score = auroc(predictions[:, i], labels[:, i])
            self.logger.experiment.add_scalar(f"{name}__roc_auc/Train", roc_score, self.current_epoch)
            
    def configure_optimizers(self): 
        optimizer = AdamW(self.parameters(), lr=2e-5)
        
        warmup_steps = self.steps_per_epoch // 3
        total_steps = self.steps_per_epoch * self.n_epochs - warmup_steps
        
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            warmup_steps,
            total_steps
        )
        
        return [optimizer], [scheduler] 

In [ ]:
class DiagnosisAndDividedObservationsClassifierExtraLayer(pl.LightningModule):
    
    def __init__(self, n_classes: int, n_diags: int, steps_per_epoch=None, n_epochs=None):
        super().__init__()
        self.bert = AutoModel.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased", return_dict=True)
        self.new_layer = nn.Linear((self.bert.config.hidden_size*4), 512)
        self.classifier = nn.Linear(512+n_diags, n_classes)
        self.steps_per_epoch = steps_per_epoch
        self.n_epochs = n_epochs
        self.criterion = nn.BCELoss()
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, 
                sne_inputs_ids, sne_attention_mask,
                st_inputs_ids, st_attention_mask,
                p_inputs_ids, p_attention_mask,
                m_inputs_ids, m_attention_mask,
                diagnosis, labels=None):
        # print(diagnosis)
        output_sne = self.bert(sne_inputs_ids, attention_mask=sne_attention_mask)
        output_st = self.bert(st_inputs_ids, attention_mask=st_attention_mask)
        output_p = self.bert(p_inputs_ids, attention_mask=p_attention_mask)
        output_m = self.bert(m_inputs_ids, attention_mask=m_attention_mask)
        # output_diagnosis = self.diagnosis_layer(torch.tensor([int(diagnosis)]))
        # diags_tensor = torch.from_numpy(diagnosis)
        output = torch.cat(
            (output_sne.pooler_output,
             output_st.pooler_output,
             output_p.pooler_output,
             output_m.pooler_output,
            ), dim=1)
        output = self.new_layer(output)
        output = self.relu(output)
        output = self.dropout(output)
        output = torch.cat(
            (
                output,
                diagnosis
            ), dim=1
        )
        output = self.classifier(output)
        output = torch.sigmoid(output)
        loss = 0
        if labels is not None:
            loss = self.criterion(output, labels)
        return loss, output
    
    def training_step(self, batch, batch_idx):
        sne_input_ids = batch['sne_input_ids']
        sne_attention_mask = batch['sne_attention_mask']
        st_input_ids = batch['st_input_ids']
        st_attention_mask = batch['st_attention_mask']
        p_input_ids = batch['p_input_ids']
        p_attention_mask = batch['p_attention_mask']
        m_input_ids = batch['m_input_ids']
        m_attention_mask = batch['m_attention_mask']
        diagnosis = batch['diagnosis']
        labels = batch['labels']
        loss, outputs = self(
            sne_input_ids, sne_attention_mask,
            st_input_ids, st_attention_mask,
            p_input_ids, p_attention_mask,
            m_input_ids, m_attention_mask,
            diagnosis, labels)
        self.log("train loss", loss, prog_bar=True, logger=True)
        return {'loss': loss, 'predictions': outputs, 'labels': labels}
    
    def validation_step(self, batch, batch_idx):
        sne_input_ids = batch['sne_input_ids']
        sne_attention_mask = batch['sne_attention_mask']
        st_input_ids = batch['st_input_ids']
        st_attention_mask = batch['st_attention_mask']
        p_input_ids = batch['p_input_ids']
        p_attention_mask = batch['p_attention_mask']
        m_input_ids = batch['m_input_ids']
        m_attention_mask = batch['m_attention_mask']
        diagnosis = batch['diagnosis']
        labels = batch['labels']
        loss, outputs = self(
            sne_input_ids, sne_attention_mask,
            st_input_ids, st_attention_mask,
            p_input_ids, p_attention_mask,
            m_input_ids, m_attention_mask,
            diagnosis, labels)
        self.log("val loss", loss, prog_bar=True, logger=True)
        return loss
    
    def test_step(self, batch, batch_idx):
        sne_input_ids = batch['sne_input_ids']
        sne_attention_mask = batch['sne_attention_mask']
        st_input_ids = batch['st_input_ids']
        st_attention_mask = batch['st_attention_mask']
        p_attention_mask = batch['p_attention_mask']
        m_input_ids = batch['m_input_ids']
        m_attention_mask = batch['m_attention_mask']
        diagnosis = batch['diagnosis']
        labels = batch['labels']
        loss, outputs = self(
            sne_input_ids, sne_attention_mask,
            st_input_ids, st_attention_mask,
            p_input_ids, p_attention_mask,
            m_input_ids, m_attention_mask,
            diagnosis, labels)
        self.log("test loss", loss, prog_bar=True, logger=True)
        return loss
    
    def training_epoch_end(self, outputs):
        labels = []
        predictions = []
        
        for output in outputs:
            for out_labels in output['labels'].detach().cpu():
                labels.append(out_labels)
                
            for out_predictions in output['predictions'].detach().cpu():
                predictions.append(out_predictions)
                
        labels = torch.stack(labels)
        predictions = torch.stack(predictions)
        
        for i, name in enumerate(Y_KEYS):
            roc_score = auroc(predictions[:, i], labels[:, i])
            self.logger.experiment.add_scalar(f"{name}__roc_auc/Train", roc_score, self.current_epoch)
            
    def configure_optimizers(self): 
        optimizer = AdamW(self.parameters(), lr=2e-5)
        
        warmup_steps = self.steps_per_epoch // 3
        total_steps = self.steps_per_epoch * self.n_epochs - warmup_steps
        
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            warmup_steps,
            total_steps
        )
        
        return [optimizer], [scheduler] 

In [ ]:
class DiagnosisAndDividedObservationsClassifierBertForDiag(pl.LightningModule):
    
    def __init__(self, n_classes: int, n_diags: int, steps_per_epoch=None, n_epochs=None):
        super().__init__()
        self.bert = AutoModel.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased", return_dict=True)
        self.bert_diag = AutoModel.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased", return_dict=True)
        self.classifier = nn.Linear(
            self.bert.config.hidden_size*4+self.bert_diag.config.hidden_size, n_classes)
        self.steps_per_epoch = steps_per_epoch
        self.n_epochs = n_epochs
        self.criterion = nn.BCELoss()
        
    def forward(self, 
                sne_inputs_ids, sne_attention_mask,
                st_inputs_ids, st_attention_mask,
                p_inputs_ids, p_attention_mask,
                m_inputs_ids, m_attention_mask,
                diagnosis_inputs_ids, diagnosis_attention_mask,
                labels=None):
        # print(diagnosis)
        output_sne = self.bert(sne_inputs_ids, attention_mask=sne_attention_mask)
        output_st = self.bert(st_inputs_ids, attention_mask=st_attention_mask)
        output_p = self.bert(p_inputs_ids, attention_mask=p_attention_mask)
        output_m = self.bert(m_inputs_ids, attention_mask=m_attention_mask)
        output_diag = self.bert_diag(diagnosis_inputs_ids, attention_mask=diagnosis_attention_mask)
        # output_diagnosis = self.diagnosis_layer(torch.tensor([int(diagnosis)]))
        # diags_tensor = torch.from_numpy(diagnosis)
        output = torch.cat(
            (output_sne.pooler_output,
             output_st.pooler_output,
             output_p.pooler_output,
             output_m.pooler_output,
             output_diag.pooler_output,
            ), dim=1)
        output = self.classifier(output)
        output = torch.sigmoid(output)
        loss = 0
        if labels is not None:
            loss = self.criterion(output, labels)
        return loss, output
    
    def training_step(self, batch, batch_idx):
        sne_input_ids = batch['sne_input_ids']
        sne_attention_mask = batch['sne_attention_mask']
        st_input_ids = batch['st_input_ids']
        st_attention_mask = batch['st_attention_mask']
        p_input_ids = batch['p_input_ids']
        p_attention_mask = batch['p_attention_mask']
        m_input_ids = batch['m_input_ids']
        m_attention_mask = batch['m_attention_mask']
        diagnosis_ids = batch['diagnosis_ids']
        diagnosis_attention_mask = batch['diagnosis_attention_mask']
        labels = batch['labels']
        loss, outputs = self(
            sne_input_ids, sne_attention_mask,
            st_input_ids, st_attention_mask,
            p_input_ids, p_attention_mask,
            m_input_ids, m_attention_mask,
            diagnosis_ids, diagnosis_attention_mask, labels)
        self.log("train loss", loss, prog_bar=True, logger=True)
        return {'loss': loss, 'predictions': outputs, 'labels': labels}
    
    def validation_step(self, batch, batch_idx):
        sne_input_ids = batch['sne_input_ids']
        sne_attention_mask = batch['sne_attention_mask']
        st_input_ids = batch['st_input_ids']
        st_attention_mask = batch['st_attention_mask']
        p_input_ids = batch['p_input_ids']
        p_attention_mask = batch['p_attention_mask']
        m_input_ids = batch['m_input_ids']
        m_attention_mask = batch['m_attention_mask']
        diagnosis_ids = batch['diagnosis_ids']
        diagnosis_attention_mask = batch['diagnosis_attention_mask']
        labels = batch['labels']
        loss, outputs = self(
            sne_input_ids, sne_attention_mask,
            st_input_ids, st_attention_mask,
            p_input_ids, p_attention_mask,
            m_input_ids, m_attention_mask,
            diagnosis_ids, diagnosis_attention_mask, labels)
        self.log("val loss", loss, prog_bar=True, logger=True)
        return loss
    
    def test_step(self, batch, batch_idx):
        sne_input_ids = batch['sne_input_ids']
        sne_attention_mask = batch['sne_attention_mask']
        st_input_ids = batch['st_input_ids']
        st_attention_mask = batch['st_attention_mask']
        p_input_ids = batch['p_input_ids']
        p_attention_mask = batch['p_attention_mask']
        m_input_ids = batch['m_input_ids']
        m_attention_mask = batch['m_attention_mask']
        diagnosis_ids = batch['diagnosis_ids']
        diagnosis_attention_mask = batch['diagnosis_attention_mask']
        labels = batch['labels']
        loss, outputs = self(
            sne_input_ids, sne_attention_mask,
            st_input_ids, st_attention_mask,
            p_input_ids, p_attention_mask,
            m_input_ids, m_attention_mask,
            diagnosis_ids, diagnosis_attention_mask, labels)
        self.log("test loss", loss, prog_bar=True, logger=True)
        return loss
    
    def training_epoch_end(self, outputs):
        labels = []
        predictions = []
        
        for output in outputs:
            for out_labels in output['labels'].detach().cpu():
                labels.append(out_labels)
                
            for out_predictions in output['predictions'].detach().cpu():
                predictions.append(out_predictions)
                
        labels = torch.stack(labels)
        predictions = torch.stack(predictions)
        
        for i, name in enumerate(Y_KEYS):
            roc_score = auroc(predictions[:, i], labels[:, i])
            self.logger.experiment.add_scalar(f"{name}__roc_auc/Train", roc_score, self.current_epoch)
            
    def configure_optimizers(self): 
        optimizer = AdamW(self.parameters(), lr=2e-5)
        
        warmup_steps = self.steps_per_epoch // 3
        total_steps = self.steps_per_epoch * self.n_epochs - warmup_steps
        
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            warmup_steps,
            total_steps
        )
        
        return [optimizer], [scheduler] 

In [ ]:
# classifier_model = JoinedObservationsClassifier(
#     n_classes=len(Y_KEYS), 
#     steps_per_epoch=len(train_dataset) // BATCH_SIZE,
#     n_epochs=N_EPOCHS
# )

In [ ]:
# # Try using gpus=['2']
# trainer = pl.Trainer(max_epochs=N_EPOCHS, gpus='1', progress_bar_refresh_rate=30)

In [ ]:
# trainer.fit(classifier_model, data_module)

In [ ]:
# classifier_model_v2 = DiagnosisAndDividedObservationsClassifier(
#     n_classes=len(Y_KEYS),
#     n_diags=len(diagnoses_keys),
#     steps_per_epoch=len(train_dataset) // BATCH_SIZE,
#     n_epochs=N_EPOCHS
# )
# classifier_model_v2 = classifier_model_v2.cuda(1)

In [ ]:
# diag_bert_classiffier = DiagnosisAndDividedObservationsClassifierBertForDiag(
#         n_classes=len(Y_KEYS),
#         n_diags=len(diagnoses_keys),
#         steps_per_epoch=len(train_dataset) // BATCH_SIZE,
#         n_epochs=N_EPOCHS
# )
# diag_bert_classiffier = diag_bert_classiffier.cuda(1)

In [ ]:
# extra_layer_classifier = DiagnosisAndDividedObservationsClassifierExtraLayer(
#         n_classes=len(Y_KEYS),
#         n_diags=len(diagnoses_keys),
#         steps_per_epoch=len(train_dataset) // BATCH_SIZE,
#         n_epochs=N_EPOCHS
# )
# extra_layer_classifier = extra_layer_classifier.cuda(2)

In [ ]:
# Try using gpus=['2']
# trainer_v2 = pl.Trainer(max_epochs=N_EPOCHS, gpus='2', progress_bar_refresh_rate=30, checkpoint_callback=False)

In [ ]:
# trainer_v2.fit(classifier_model_v2, data_module_200)
# trainer_v2.save_checkpoint('/research/jamunoz/last_checkpoint_diagnosis_divided_obs_200.ckpt')

In [ ]:
# trainer_v2.fit(classifier_model_v2, data_module_300)
# trainer_v2.save_checkpoint('/research/jamunoz/last_checkpoint_diagnosis_divided_obs_300.ckpt')

In [ ]:
# trainer_v2.fit(classifier_model_v2, data_module_400)
# trainer_v2.save_checkpoint('/research/jamunoz/last_checkpoint_diagnosis_divided_obs_400.ckpt')

In [ ]:
# trainer_v2.fit(classifier_model_v2, data_module_500)
# trainer_v2.save_checkpoint('/research/jamunoz/last_checkpoint_diagnosis_divided_obs_500.ckpt')hamming_loss

In [ ]:
# trainer_v2.fit(diag_bert_classiffier, data_module_200)
# trainer_v2.save_checkpoint('/research/jamunoz/last_checkpoint_diagnosis_divided_obs_200_bert_diag.ckpt')

In [ ]:
# trainer_v2.fit(diag_bert_classiffier, data_module_300)
# trainer_v2.save_checkpoint('/research/jamunoz/last_checkpoint_diagnosis_divided_obs_300_bert_diag.ckpt')

In [ ]:
# trainer_v2.fit(diag_bert_classiffier, data_module_400)
# trainer_v2.save_checkpoint('/research/jamunoz/last_checkpoint_diagnosis_divided_obs_400_bert_diag.ckpt')

In [ ]:
# trainer_v2.fit(diag_bert_classiffier, data_module_500)
# trainer_v2.save_checkpoint('/research/jamunoz/last_checkpoint_diagnosis_divided_obs_500_bert_diag.ckpt')

In [ ]:
# trainer_v2.fit(extra_layer_classifier, mini_400_module)
# trainer_v2.save_checkpoint('/research/jamunoz/last_checkpoint_diagnosis_divided_obs_400_extra_layer.ckpt')

## Predictions

In [ ]:
def get_predictions(trained_model, test_dataset):
    predictions = []
    labels = []
    for row in test_dataset:
        _, raw_prediction = trained_model(
            row['sne_input_ids'].reshape(1, len(row['sne_input_ids'])),
            row['sne_attention_mask'].reshape(1, len(row['sne_attention_mask'])),
            row['st_input_ids'].reshape(1, len(row['st_input_ids'])),
            row['st_attention_mask'].reshape(1, len(row['st_attention_mask'])),
            row['p_input_ids'].reshape(1, len(row['p_input_ids'])),
            row['p_attention_mask'].reshape(1, len(row['p_attention_mask'])),
            row['m_input_ids'].reshape(1, len(row['m_input_ids'])),
            row['m_attention_mask'].reshape(1, len(row['m_attention_mask'])),
            torch.from_numpy(row['diagnosis']).reshape(1, len(row['diagnosis']))
        )
#         print(raw_prediction)
        prediction = [1 if pred > 0.5 else 0 for pred in raw_prediction[0]]
        predictions.append(prediction)
        labels.append(row['labels'].numpy())
    return predictions, labels

In [ ]:
def get_text_diag_predictions(trained_model, test_dataset):
    predictions = []
    labels = []
    for row in test_dataset:
        _, raw_prediction = trained_model(
            row['sne_input_ids'].reshape(1, len(row['sne_input_ids'])),
            row['sne_attention_mask'].reshape(1, len(row['sne_attention_mask'])),
            row['st_input_ids'].reshape(1, len(row['st_input_ids'])),
            row['st_attention_mask'].reshape(1, len(row['st_attention_mask'])),
            row['p_input_ids'].reshape(1, len(row['p_input_ids'])),
            row['p_attention_mask'].reshape(1, len(row['p_attention_mask'])),
            row['m_input_ids'].reshape(1, len(row['m_input_ids'])),
            row['m_attention_mask'].reshape(1, len(row['m_attention_mask'])),
            row['diagnosis_ids'].reshape(1, len(row['diagnosis_ids'])),
            row['diagnosis_attention_mask'].reshape(1, len(row['diagnosis_attention_mask'])),
        )
#         print(raw_prediction)
        prediction = [1 if pred > 0.5 else 0 for pred in raw_prediction[0]]
        predictions.append(prediction)
        labels.append(row['labels'].numpy())
    return predictions, labels

In [ ]:
# get label of test set
y_true = [row['labels'].numpy() for row in data_module_200.test_dataset]

In [ ]:
# trainer_v2.save_checkpoint('/research/jamunoz/last_checkpoint_diagnosis_divided_obs.ckpt')
# classifier_model_v2 = DiagnosisAndDividedObservationsClassifier.load_from_checkpoint(
#     '/research/jamunoz/last_checkpoint_diagnosis_divided_obs.ckpt', n_classes=len(Y_KEYS), n_diags=len(diagnoses_keys))
# classifier_model_v2.freeze()

In [ ]:
# classifier_model = DiagnosisAndDividedObservationsClassifier.load_from_checkpoint(
#     '/research/jamunoz/last_checkpoint_diagnosis_divided_obs_200.ckpt', n_classes=len(Y_KEYS), n_diags=len(diagnoses_keys))
# classifier_model.freeze()
# y_pred_200, y_true_200 = get_predictions(classifier_model, data_module_200.test_dataset)
# del classifier_model

In [ ]:
# classifier_model = DiagnosisAndDividedObservationsClassifier.load_from_checkpoint(
#     '/research/jamunoz/last_checkpoint_diagnosis_divided_obs_300.ckpt', n_classes=len(Y_KEYS), n_diags=len(diagnoses_keys))
# classifier_model.freeze()
# y_pred_300, y_true_300 = get_predictions(classifier_model, data_module_300.test_dataset)
# del classifier_model

In [ ]:
# classifier_model = DiagnosisAndDividedObservationsClassifier.load_from_checkpoint(
#     '/research/jamunoz/last_checkpoint_diagnosis_divided_obs_400.ckpt', n_classes=len(Y_KEYS), n_diags=len(diagnoses_keys))
# classifier_model.freeze()
# y_pred_400, y_true_400 = get_predictions(classifier_model, data_module_400.test_dataset)
# del classifier_model

In [ ]:
# classifier_model = DiagnosisAndDividedObservationsClassifier.load_from_checkpoint(
#     '/research/jamunoz/last_checkpoint_diagnosis_divided_obs_500.ckpt', n_classes=len(Y_KEYS), n_diags=len(diagnoses_keys))
# classifier_model.freeze()
# y_pred_500, y_true_500 = get_predictions(classifier_model, data_module_500.test_dataset)
# del classifier_model

In [ ]:
# classifier_model = DiagnosisAndDividedObservationsClassifierBertForDiag.load_from_checkpoint(
#     '/research/jamunoz/last_checkpoint_diagnosis_divided_obs_200_bert_diag.ckpt', n_classes=len(Y_KEYS), n_diags=len(diagnoses_keys))
# classifier_model.freeze()
# y_pred_200_bert_diag, _ = get_text_diag_predictions(classifier_model, data_module_200.test_dataset)
# del classifier_model
# with open('preds_200_bert_diag.npy', 'wb') as f:
#     np.save(f, y_pred_200_bert_diag)

In [ ]:
# classifier_model = DiagnosisAndDividedObservationsClassifierBertForDiag.load_from_checkpoint(
#     '/research/jamunoz/last_checkpoint_diagnosis_divided_obs_300_bert_diag.ckpt', n_classes=len(Y_KEYS), n_diags=len(diagnoses_keys))
# classifier_model.freeze()
# y_pred_300_bert_diag, _ = get_text_diag_predictions(classifier_model, data_module_300.test_dataset)
# del classifier_model
# with open('preds_300_bert_diag.npy', 'wb') as f:
#     np.save(f, y_pred_300_bert_diag)
    
# classifier_model = DiagnosisAndDividedObservationsClassifierBertForDiag.load_from_checkpoint(
#     '/research/jamunoz/last_checkpoint_diagnosis_divided_obs_400_bert_diag.ckpt', n_classes=len(Y_KEYS), n_diags=len(diagnoses_keys))
# classifier_model.freeze()
# y_pred_400_bert_diag, _ = get_text_diag_predictions(classifier_model, data_module_400.test_dataset)
# del classifier_model
# with open('preds_400_bert_diag.npy', 'wb') as f:
#     np.save(f, y_pred_400_bert_diag)
    
# classifier_model = DiagnosisAndDividedObservationsClassifierBertForDiag.load_from_checkpoint(
#     '/research/jamunoz/last_checkpoint_diagnosis_divided_obs_500_bert_diag.ckpt', n_classes=len(Y_KEYS), n_diags=len(diagnoses_keys))
# classifier_model.freeze()
# y_pred_500_bert_diag, _ = get_text_diag_predictions(classifier_model, data_module_500.test_dataset)
# del classifier_model
# with open('preds_500_bert_diag.npy', 'wb') as f:
#     np.save(f, y_pred_500_bert_diag)

In [ ]:
# print('haciendo 200')
# classifier_model = DiagnosisAndDividedObservationsClassifierExtraLayer.load_from_checkpoint(
#     '/research/jamunoz/last_checkpoint_diagnosis_divided_obs_200_extra_layer.ckpt', n_classes=len(Y_KEYS), n_diags=len(diagnoses_keys))
# classifier_model.freeze()
# y_pred_200_extra_layer, _ = get_predictions(classifier_model, data_module_200.test_dataset)
# del classifier_model
# with open('preds_200_extra_layer.npy', 'wb') as f:
#     np.save(f, y_pred_200_extra_layer)

# print('haciendo 300')
# classifier_model = DiagnosisAndDividedObservationsClassifierExtraLayer.load_from_checkpoint(
#     '/research/jamunoz/last_checkpoint_diagnosis_divided_obs_300_extra_layer.ckpt', n_classes=len(Y_KEYS), n_diags=len(diagnoses_keys))
# classifier_model.freeze()
# y_pred_300_extra_layer, _ = get_predictions(classifier_model, data_module_300.test_dataset)
# del classifier_model
# with open('preds_300_extra_layer.npy', 'wb') as f:
#     np.save(f, y_pred_300_extra_layer)

# print('haciendo 400')
# classifier_model = DiagnosisAndDividedObservationsClassifierExtraLayer.load_from_checkpoint(
#     '/research/jamunoz/last_checkpoint_diagnosis_divided_obs_400_extra_layer.ckpt', n_classes=len(Y_KEYS), n_diags=len(diagnoses_keys))
# classifier_model.freeze()
# y_pred_400_extra_layer, _ = get_predictions(classifier_model, data_module_400.test_dataset)
# del classifier_model
# with open('preds_400_extra_layer.npy', 'wb') as f:
#     np.save(f, y_pred_400_extra_layer)

In [ ]:
# with open('preds_200.npy', 'wb') as f:
#     np.save(f, y_pred_200)
y_pred_200 = np.load('preds_200.npy')

# with open('preds_300.npy', 'wb') as x:
#     np.save(x, y_pred_300)
y_pred_300 = np.load('preds_300.npy')

# with open('preds_400.npy', 'wb') as y:
#     np.save(y, y_pred_400)
y_pred_400 = np.load('preds_400.npy')

# with open('preds_500.npy', 'wb') as y:
#     np.save(y, y_pred_500)
y_pred_500 = np.load('preds_500.npy')

y_pred_200_bert_diag = np.load('preds_200_bert_diag.npy')

y_pred_300_bert_diag = np.load('preds_300_bert_diag.npy')

y_pred_400_bert_diag = np.load('preds_400_bert_diag.npy')

y_pred_500_bert_diag = np.load('preds_500_bert_diag.npy')


y_pred_200_extra_layer = np.load('preds_200_extra_layer.npy')

y_pred_300_extra_layer = np.load('preds_300_extra_layer.npy')

y_pred_400_extra_layer = np.load('preds_400_extra_layer.npy')

In [ ]:
def comp(arr_of_arr1, arr_of_arr2):
    if len(arr_of_arr1) != len(arr_of_arr2):
        return False
    else:
        for (i, arr) in enumerate(arr_of_arr1):
            if len(arr) != len(arr_of_arr2[i]):
                return False
            else:
                for (j, el) in enumerate(arr):
                    if el != arr_of_arr2[i][j]:
                        return False
    return True

In [ ]:
print('max token 200 OHE diagnosis Single BETO')
print('Accuracy: ', accuracy_score(y_true, y_pred_200))
print('Micro F1: ', f1_score(y_true, y_pred_200, average='micro'))
print('Macro F1: ', f1_score(y_true, y_pred_200, average='macro'))
print('Weighted F1: ', f1_score(y_true, y_pred_200, average='weighted'))
print('Hamming Loss: ', hamming_loss(y_true, y_pred_200))

In [ ]:
print('max token 300 OHE diagnosis Single BETO')
print('Accuracy: ', accuracy_score(y_true, y_pred_300))
print('Micro F1: ', f1_score(y_true, y_pred_300, average='micro'))
print('Macro F1: ', f1_score(y_true, y_pred_300, average='macro'))
print('Weighted F1: ', f1_score(y_true, y_pred_300, average='weighted'))
print('Hamming Loss: ', hamming_loss(y_true, y_pred_300))

In [ ]:
print('max token 400 OHE diagnosis Single BETO')
print('Accuracy: ', accuracy_score(y_true, y_pred_400))
print('Micro F1: ', f1_score(y_true, y_pred_400, average='micro'))
print('Macro F1: ', f1_score(y_true, y_pred_400, average='macro'))
print('Weighted F1: ', f1_score(y_true, y_pred_400, average='weighted'))
print('Hamming Loss: ', hamming_loss(y_true, y_pred_400))

In [ ]:
print('max token 500 OHE diagnosis Single BETO')
print('Accuracy: ', accuracy_score(y_true, y_pred_500))
print('Micro F1: ', f1_score(y_true, y_pred_500, average='micro'))
print('Macro F1: ', f1_score(y_true, y_pred_500, average='macro'))
print('Weighted F1: ', f1_score(y_true, y_pred_500, average='weighted'))
print('Hamming Loss: ', hamming_loss(y_true, y_pred_500))

In [ ]:
print('max token 200 BETO diagnosis Single BETO')
print('Accuracy: ', accuracy_score(y_true, y_pred_200_bert_diag))
print('Micro F1: ', f1_score(y_true, y_pred_200_bert_diag, average='micro'))
print('Macro F1: ', f1_score(y_true, y_pred_200_bert_diag, average='macro'))
print('Weighted F1: ', f1_score(y_true, y_pred_200_bert_diag, average='weighted'))
print('Hamming Loss: ', hamming_loss(y_true, y_pred_200_bert_diag))

In [ ]:
print('max token 300 BETO diagnosis Single BETO')
print('Accuracy: ', accuracy_score(y_true, y_pred_300_bert_diag))
print('Micro F1: ', f1_score(y_true, y_pred_300_bert_diag, average='micro'))
print('Macro F1: ', f1_score(y_true, y_pred_300_bert_diag, average='macro'))
print('Weighted F1: ', f1_score(y_true, y_pred_300_bert_diag, average='weighted'))
print('Hamming Loss: ', hamming_loss(y_true, y_pred_300_bert_diag))

In [ ]:
print('max token 400 BETO diagnosis Single BETO')
print('Accuracy: ', accuracy_score(y_true, y_pred_400_bert_diag))
print('Micro F1: ', f1_score(y_true, y_pred_400_bert_diag, average='micro'))
print('Macro F1: ', f1_score(y_true, y_pred_400_bert_diag, average='macro'))
print('Weighted F1: ', f1_score(y_true, y_pred_400_bert_diag, average='weighted'))
print('Hamming Loss: ', hamming_loss(y_true, y_pred_400_bert_diag))

In [ ]:
print('max token 500 BETO diagnosis Single BETO')
print('Accuracy: ', accuracy_score(y_true, y_pred_500_bert_diag))
print('Micro F1: ', f1_score(y_true, y_pred_500_bert_diag, average='micro'))
print('Macro F1: ', f1_score(y_true, y_pred_500_bert_diag, average='macro'))
print('Weighted F1: ', f1_score(y_true, y_pred_500_bert_diag, average='weighted'))
print('Hamming Loss: ', hamming_loss(y_true, y_pred_500_bert_diag))

In [ ]:
print('max token 200 OHE diagnosis Single BETO Additional layer')
print('Accuracy: ', accuracy_score(y_true, y_pred_200_extra_layer))
print('Micro F1: ', f1_score(y_true, y_pred_200_extra_layer, average='micro'))
print('Macro F1: ', f1_score(y_true, y_pred_200_extra_layer, average='macro'))
print('Weighted F1: ', f1_score(y_true, y_pred_200_extra_layer, average='weighted'))
print('Hamming Loss: ', hamming_loss(y_true, y_pred_200_extra_layer))

In [ ]:
print('max token 300 OHE diagnosis Single BETO Additional layer')
print('Accuracy: ', accuracy_score(y_true, y_pred_300_extra_layer))
print('Micro F1: ', f1_score(y_true, y_pred_300_extra_layer, average='micro'))
print('Macro F1: ', f1_score(y_true, y_pred_300_extra_layer, average='macro'))
print('Weighted F1: ', f1_score(y_true, y_pred_300_extra_layer, average='weighted'))
print('Hamming Loss: ', hamming_loss(y_true, y_pred_300_extra_layer))

In [ ]:
print('max token 400 OHE diagnosis Single BETO Additional layer')
print('Accuracy: ', accuracy_score(y_true, y_pred_400_extra_layer))
print('Micro F1: ', f1_score(y_true, y_pred_400_extra_layer, average='micro'))
print('Macro F1: ', f1_score(y_true, y_pred_400_extra_layer, average='macro'))
print('Weighted F1: ', f1_score(y_true, y_pred_400_extra_layer, average='weighted'))
print('Hamming Loss: ', hamming_loss(y_true, y_pred_400_extra_layer))